# Train a customer RNN for time series forecasting

## Import libraries

In [1]:
import numpy
import pandas as pd
import math
import random
import keras
from keras.models import Sequential
from keras.models import load_model, Model
from keras.layers import Dense, Input, GRU, concatenate, Activation
from keras.optimizers import Adam
from keras.layers.core import Dense, Dropout, Activation
from collections import deque
from pprint import pprint
import csv
import h5py

#from customRNN import trainRNN


Using TensorFlow backend.


## Define hyperparameters

In [2]:
datapath = './src/datasets/AMZN_train.csv'
lag = 10
horiz = 10
nfeatures = 1
nunits = 64            # Number of GRUs in recurrent layer
nepochs = 100         # Number of epochs (originally 1000)
d = 0.2                # Percent of neurons to drop at each epoch
optimizer = 'adam'     # Optimization algorithm (also tried rmsprop)
activ = 'elu'          # Activation function for neurons (elu faster than sigmoid)
activr = 'hard_sigmoid'# Activation function for recurrent layer
activd = 'linear'      # Dense layer's activation function
lossmetric = 'mean_absolute_error'  # Loss function for gradient descent
verbose = True        # Whether or not to list results of each epoch

## Import data and train 

In [3]:
# Import training data
df = pd.read_csv(datapath)

# Prepare data
df["Adjclose"] = df.Close # Moving close price to last column
df.drop(['Date','Close','Adj Close'], 1, inplace=True)
#df = df.diff() 

data = df.as_matrix()
lags = []
horizons = []
nsample = len(data) - lag - horiz  # Number of time series (Number of sample in 3D)

for i in range(nsample):
                    lags.append(data[i: i + lag , -nfeatures:])
                    horizons.append(data[i + lag : i + lag + horiz, -1])
            
lags = numpy.array(lags)
horizons = numpy.array(horizons)
print("Number of horizons: ", len(horizons))
lags = numpy.reshape(lags, (lags.shape[0], lags.shape[1], nfeatures))


# Design RNN architecture
rnn_in = Input(shape = (lag, nfeatures), dtype = 'float32', name = 'rnn_in')
rnn_gru = GRU(units = nunits, return_sequences = False, activation = activ, recurrent_activation = activr, dropout = d)(rnn_in)
rnn_out = Dense(horiz, activation = activd, name = 'rnn_out')(rnn_gru)
model = Model(inputs = [rnn_in], outputs = [rnn_out])
model.compile(optimizer = optimizer, loss = lossmetric)

# Train model
fcst = model.fit({'rnn_in': lags},{'rnn_out': horizons}, epochs=nepochs,verbose=verbose)

# Save model
model.save('./src/custom-rnn.h5')
  


Number of horizons:  1743





/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





1743/1743 [==============================] - 3s 2ms/step - loss: 150.7115
Epoch 2/100
1743/1743 [==============================] - 1s 527us/step - loss: 53.8263
Epoch 3/100
1743/1743 [==============================] - 1s 540us/step - loss: 38.9428
Epoch 4/100
1743/1743 [==============================] - 1s 571us/step - loss: 16.8841
Epoch 5/100
1743/1743 [==============================] - 1s 561us/step - loss: 10.9148
Epoch 6/100
1743/1743 [==============================] - 1s 547us/step - loss: 10.3776
Epoch 7/100
1743/1743 [==============================] - 1s 558us/step - loss: 10.0706
Epoch 8/100
1743/1743 [==============================] - 1s 576us/step - loss: 9.2295
Epoch 9/100
1743/1743 [==============================] - 1s 549us/step - loss: 10.6417


## Prepare static set of forecasts for experimentation purpose

In [4]:
datapath = './src/datasets/AMZN_test.csv'
horizonpath = './src/datasets/AMZN_test_horizons.npy'

# Read the asset price time series 
t = 0
vec = []
payload = []
lines = open(datapath, "r").read().splitlines()


for line in lines[1:]:
    vec.append(float(line.split(",")[4]))
    d = t - lag + 1
    block = vec[d:t + 1] if d >= 0 else -d * [vec[0]] + vec[0:t + 1] # pad with t0
    payload.append(block)
    t += 1

payload = numpy.array(payload)
payload = numpy.reshape(payload, (payload.shape[0], payload.shape[1], 1))
print('Payload array shape: {}'.format(payload.shape))
horiz = model.predict(payload)
print('Forecasted horizons array shape: {}'.format(horiz.shape))

numpy.save(horizonpath, numpy.array(horiz)) # fmt="%s")


Payload array shape: (755, 10, 1)
Forecasted horizons array shape: (755, 10)


## Annexes / Experiment sandbox

In [5]:
test = numpy.load(horizonpath)

In [6]:
# Function to return sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

print(test[0])
block = test[0]
res = []
for i in range(lag - 1):
    res.append(sigmoid(block[i + 1] - block[i]))

print(res)

[244.7038  244.57945 247.31195 246.60397 245.01344 247.04045 245.96434
 246.94934 247.62444 245.81914]
[0.4689540228637265, 0.9389172684410546, 0.33004593802222965, 0.16930928708800247, 0.8836037149684083, 0.2542427094209506, 0.7280993190979925, 0.6626429860704051, 0.14120738376376912]
